## 第9章　カテゴリ型

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

### 9-1　カテゴリ型への変換

カテゴリ型は、カテゴリ値をマスタデータ（'man', 'woman'など）とインデックスデータ（0, 1など）に分けて保持している。

In [2]:
customer_tb = pd.read_csv('customer.csv')

customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


In [3]:
customer_tb[['sex_is_man']] = (customer_tb[['sex']] == 'man').astype('bool')

customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man
0,c_1,41,man,35.092193,136.512347,True
1,c_2,38,man,35.325076,139.410551,True
2,c_3,49,woman,35.120543,136.511179,False
3,c_4,43,man,43.034868,141.240314,True
4,c_5,31,man,35.102661,136.523797,True


次の方法では、カテゴリ型に変換している。

In [4]:
customer_tb['sex_c'] = pd.Categorical(customer_tb['sex'], categories=['man', 'woman'])

customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c
0,c_1,41,man,35.092193,136.512347,True,man
1,c_2,38,man,35.325076,139.410551,True,man
2,c_3,49,woman,35.120543,136.511179,False,woman
3,c_4,43,man,43.034868,141.240314,True,man
4,c_5,31,man,35.102661,136.523797,True,man


In [5]:
customer_tb['sex_c'] = customer_tb['sex_c'].astype('category')

pd.DataFrame(customer_tb.dtypes, columns=['dtype'])

,dtype
customer_id,object
age,int64
sex,object
home_latitude,float64
home_longitude,float64
sex_is_man,bool
sex_c,category


インデックスデータはcodesに格納されている。0がman、1がwomanに対応している。

In [6]:
pd.DataFrame(customer_tb['sex_c'].cat.codes, columns=['index_data']).head()

,index_data
0,0
1,0
2,1
3,0
4,0


In [7]:
pd.DataFrame(customer_tb['sex_c'].cat.categories, columns=['Data'])

,Data
0,man
1,woman


### 9-2　ダミー変数化

In [8]:
customer_tb['sex'] = pd.Categorical(customer_tb['sex'])

dummy_vars = pd.get_dummies(customer_tb['sex'], drop_first=True)
dummy_vars.head()

,woman
0,0
1,0
2,1
3,0
4,0


ダミー変数を1つ減らすと、少ないデータ量で保持できる反面、データ分析の際に不便になってしまうことがある。<br>
このような場合は、ダミー変数を減らさない方が良い。

In [9]:
dummy_vars = pd.get_dummies(customer_tb['sex'], drop_first=False)
dummy_vars.head()

,man,woman
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


### 9-3　カテゴリ値の集約

In [10]:
customer_tb = pd.read_csv('customer.csv')

customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


最初に、np.floor関数の補足をしておく。<br>
10で割って少数点のある数値にした後に、np.floorを適用し10を掛けることで、1の位を切り捨てできる。

In [11]:
x = 46
x = x / 10
x = np.floor(x)
print(x)

x = x * 10
print(x)

4.0
40.0


カテゴリ値が多すぎたり、少ないカテゴリ値が存在すると、過学習に陥りやすくなるので<br>
似たようなカテゴリ値はまとめてしまう（集約する）した方が良い。

In [12]:
customer_tb['age_rank'] = pd.Categorical(np.floor(customer_tb['age']/10)*10)

customer_tb['age_rank'].cat.add_categories(['60以上'], inplace=True)
customer_tb.loc[customer_tb['age_rank'].isin([60.0, 70.0, 80.0]), 'age_rank'] = '60以上'

customer_tb['age_rank'].cat.remove_unused_categories(inplace=True)

customer_tb.head(10)

,customer_id,age,sex,home_latitude,home_longitude,age_rank
0,c_1,41,man,35.092193,136.512347,40
1,c_2,38,man,35.325076,139.410551,30
2,c_3,49,woman,35.120543,136.511179,40
3,c_4,43,man,43.034868,141.240314,40
4,c_5,31,man,35.102661,136.523797,30
5,c_6,52,man,34.440768,135.390487,50
6,c_7,50,man,43.015758,141.231321,50
7,c_8,65,woman,38.201268,140.465961,60以上
8,c_9,36,woman,33.322800,130.330689,30
9,c_10,34,woman,34.290414,132.302601,30


このように、カテゴリ型の集約（カテゴリ値の付与）については<br>
pd.Categorical関数やcat.add.categories関数を使用すると良い。

### 9-4　カテゴリ値の組み合わせ

In [13]:
customer_tb = pd.read_csv('customer.csv')

customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


pd.Categorical関数は、新たなカテゴリ変数（カテゴリカル）を作る際に使用する。

In [14]:
customer_tb['sex_and_age'] = pd.Categorical(customer_tb[['sex', 'age']]
                                            .apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1] / 10) * 10), axis=1))

customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_and_age
0,c_1,41,man,35.092193,136.512347,man_40.0
1,c_2,38,man,35.325076,139.410551,man_30.0
2,c_3,49,woman,35.120543,136.511179,woman_40.0
3,c_4,43,man,43.034868,141.240314,man_40.0
4,c_5,31,man,35.102661,136.523797,man_30.0


x[0]には'sex'、x[1]には'age'が入っている。<br>
また、カラム（sex_and_age）に出力されているデータの形となるように、formatで出力形式を調整している。

### 9-5　カテゴリ型の数値化

In [15]:
production = pd.read_csv('production.csv')

print(production.shape)
production.head()

(1000, 4)


,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


fault_flgがTrueのものを取り出し、グループ化し、障害数をカウントする。

In [16]:
pd.DataFrame(production.query('fault_flg')).head(10)

,type,length,thickness,fault_flg
28,A,131.849213,4.721601,True
43,E,110.243125,19.164371,True
47,A,75.302562,8.513450,True
56,C,205.879240,24.903008,True
57,B,140.825485,2.833781,True
61,A,76.057115,3.231957,True
78,E,111.992565,14.126904,True
102,C,183.058351,2.628463,True
119,D,83.803035,1.109715,True
121,C,222.060118,20.157417,True


In [17]:
fault_cnt_per_type = production.query('fault_flg').groupby('type')['fault_flg'].count()

pd.DataFrame(fault_cnt_per_type)

,fault_flg
type,
A,11
B,6
C,16
D,7
E,12


fault_flgがTrue・False問わず、グループ化し、製造数をカウントする。

In [18]:
type_cnt = production.groupby('type')['fault_flg'].count()

pd.DataFrame(type_cnt)

,fault_flg
type,
A,202
B,175
C,211
D,215
E,197


下記のようなlambdaの使い方がある。

x[0]：production['length']<br>
x[1]：production['thickness']

axis=1で列方向（行ごと）に計算をしている。

In [19]:
production['sample'] = production[['length', 'thickness']].apply(lambda x: x[0] * x[1], axis=1)

production.head()

,type,length,thickness,fault_flg,sample
0,E,274.027383,40.241131,False,11027.171904
1,D,86.319269,16.906715,False,1459.375241
2,E,123.940388,1.018462,False,126.228575
3,B,175.554886,16.414924,False,2881.720146
4,B,244.934740,29.061081,False,7118.068257


type別のfault_flgのカウント数を、productionに割り当てる。

In [20]:
pd.DataFrame(fault_cnt_per_type[production['type']]).head(10)

,fault_flg
type,
E,12
D,7
E,12
B,6
B,6
B,6
C,16
A,11
C,16


In [21]:
pd.DataFrame(production['fault_flg']).astype(int).head(10)

,fault_flg
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


type別の製品数を、productionに割り当てる。

In [22]:
pd.DataFrame(type_cnt[production['type']]).head(10)

,fault_flg
type,
E,197
D,215
E,197
B,175
B,175
B,175
C,211
A,202
C,211


In [23]:
pd.DataFrame(np.ones(10).astype(int))

,0
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


上記4つの値を用いて、平均障害率を求める。

In [24]:
production['type_fault_rate'] = production[['type', 'fault_flg']] \
.apply(lambda x: (fault_cnt_per_type[x[0]] - int(x[1])) / (type_cnt[x[0]] -1), axis=1)

production.head()

,type,length,thickness,fault_flg,sample,type_fault_rate
0,E,274.027383,40.241131,False,11027.171904,0.061224
1,D,86.319269,16.906715,False,1459.375241,0.032710
2,E,123.940388,1.018462,False,126.228575,0.061224
3,B,175.554886,16.414924,False,2881.720146,0.034483
4,B,244.934740,29.061081,False,7118.068257,0.034483


x[0]：production['type']<br>
x[1]：production['fault_flg']

axis=1で列方向（行ごと）に計算をしている。

平均障害率は、各値ごとのtypeの障害数から障害の有無（0, 1）を引き、各値ごとのtypeの製造数から1を引いたもので割る。<br>
具体的な計算式は記載されているので、ここでは実行したコードの理解ができていれば良い。

### 9-6　カテゴリ型の補完

KNN（k近傍法）を用いて欠損値の予測を行い、補完する。<br>
**欠損値の近くのデータを調べ、多数決で補完値を決める。**

In [29]:
production_missc_tb = pd.read_csv('production_missing_category.csv')

production_missc_tb.replace('None', np.nan, inplace=True)

print(production_missc_tb.isnull().sum())
print(production_missc_tb.shape)
production_missc_tb.head()

type         100
length         0
thickness      0
fault_flg      0
dtype: int64
(1000, 4)


,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


typeに欠損値を持つデータが100行ある。<br>
subsetを用いて、type列に欠損値がある行を削除する。

In [28]:
train = production_missc_tb.dropna(subset=['type'])

print(train.shape)
print(train.isnull().sum())
train.head()

(900, 4)
type         0
length       0
thickness    0
fault_flg    0
dtype: int64


,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


DataFrame.index.defferenceで、trainで選択されていない残りのデータのインデックスを指定する。

In [27]:
test = production_missc_tb.loc[production_missc_tb.index.difference(train.index), :]

print(test.shape)
print(test.isnull().sum())
test.head()

(100, 4)
type         100
length         0
thickness      0
fault_flg      0
dtype: int64


,type,length,thickness,fault_flg
8,NaN,276.386631,29.899611,False
26,NaN,263.844324,34.664251,False
30,NaN,129.364736,21.346752,False
36,NaN,203.378972,30.286454,False
41,NaN,157.463166,11.166165,False


In [32]:
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train[['length', 'thickness']], train['type'])

test['type'] = kn.predict(test[['length', 'thickness']])

In [34]:
print(test.isnull().sum())
test.head()

type         0
length       0
thickness    0
fault_flg    0
dtype: int64


,type,length,thickness,fault_flg
8,E,276.386631,29.899611,False
26,E,263.844324,34.664251,False
30,E,129.364736,21.346752,False
36,A,203.378972,30.286454,False
41,E,157.463166,11.166165,False
